In [1]:
import torch


device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

device

'cuda'

In [2]:
usr = "char"

# Tengo GPU pero no cuda compatible, me rompe cuando intento correr con GPU
if usr == "nico":
    device="cpu"
    num_workers = 4
    
if usr == "char":
    num_workers = 0

In [3]:
from catdog.dataset import CatDogDataset

In [4]:
DATA_DIR = '../data/'

In [9]:
import pandas as pd
import torchvision.transforms as tvt

from catdog.utils.image import BBoxIdentityWrapper, RandomHorizontalFlipBBox, RandomVerticalFlipBBox


identity_transforms = [
    tvt.RandomGrayscale(),
    tvt.ColorJitter(brightness=10, contrast=10, saturation=10, hue=[-0.5, 0.5]),
    tvt.RandomInvert(p=.1),
    tvt.RandomPosterize(bits=7, p=.1), # keeps 7 bits for each channel
    tvt.RandomAutocontrast(p=.1),
    #tvt.RandomAdjustSharpness(sharpness_factor=1.25, p=0.1),  # sharpens
    #tvt.RandomAdjustSharpness(sharpness_factor=.75, p=0.1),  # blurs
    #tvt.RandomSolarize(threshold=250, p=0.1),
    #tvt.RandomEqualize(p=0.1)
]
wrapped_identity_transforms = [BBoxIdentityWrapper(t) for t in identity_transforms]
flipper_transforms = [RandomHorizontalFlipBBox(), RandomVerticalFlipBBox()]


# fisura way to use tvt.Compose on (img, bbox) pairs
class BBoxCompose(tvt.Compose):
    
    calls = 0
    
    def __call__(self, img, bbox):
        BBoxCompose.calls += 1
        try:
            if BBoxCompose.calls == 3:
                # import ipdb; ipdb.set_trace()
                pass
            for t in self.transforms:
                img, bbox = t(img, bbox)
        except TypeError as e:
            print(e)
            import ipdb; ipdb.set_trace()
            pass
        return img, bbox

# composed_transforms = BBoxCompose(wrapped_identity_transforms + flipper_transforms)
# composed_transforms = BBoxCompose(flipper_transforms)
composed_transforms = BBoxCompose(flipper_transforms)

train_set_pd = pd.read_csv(DATA_DIR + 'train.csv')
train_set_torch = CatDogDataset(train_set_pd, transformations=composed_transforms)
print(f"Loaded train set with length {len(train_set_torch)}")

test_set_pd = pd.read_csv(DATA_DIR + 'test.csv')
test_set_torch = CatDogDataset(test_set_pd)
print(f"Loaded test set with length {len(test_set_torch)}")

Loaded train set with length 2948
Loaded test set with length 738


In [10]:
batch_size = 82
train_loader = torch.utils.data.DataLoader(train_set_torch, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_set_torch, batch_size=batch_size, shuffle=False, num_workers=num_workers)


In [11]:
import numpy as np


seed = 142
torch.manual_seed(seed)
np.random.seed(seed)

#### test image

import numpy as np
from PIL import Image


test_img_path = DATA_DIR + 'images/' + cat_dog_df["file"].iloc[0]
test_img = Image.open(test_img_path).convert("RGB")

In [12]:
from catdog.models.feedforward import MLPClassifier
from catdog.models.convolutional import ConvolutionalClassifier


# model_kwargs = {'input_size': 500*500*3, 'hidden_sizes': (500, 350, 256, 128, 50, 10)}
# model = MLPClassifier(**model_kwargs)

lr_str = '2e-4' 
dropout_p = 0
model_kwargs = {
    'input_shape': (3, 500, 500), 
    'dropout_p': dropout_p, 
    'optimizer_params':{'lr': float(lr_str)}
}
model = ConvolutionalClassifier(**model_kwargs)
model.to(device)

/usr/local/lib/python3.6/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


ConvolutionalClassifier(
  (AUROC): AUROC()
  (Precision): Precision()
  (Recall): Recall()
  (model): Sequential(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (relu1): ReLU()
    (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (batch_norm_1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (relu2): ReLU()
    (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (batch_norm_2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (relu3): ReLU()
    (pool3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (batch_norm_3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (flatten): Flatten(start_dim=1, end_d

In [13]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

seed_everything(142, workers=True)

if dropout_p:
    reg = f'dropout_{dropout_p}_'
else:
    reg = 'batch_norm_'

exp_name = f'CNN_classifier_3_layers_with_max_pooling_all_transforms_{reg}batch_size_{batch_size}_lr_{lr_str}'
print(exp_name)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='./lightning_checkpoints/',
    filename=exp_name,
    mode='min',
    save_top_k=3
)

logger = TensorBoardLogger("tb_logs", name=exp_name)

trainer = Trainer(gpus=int(device=='cuda'), # hacky way to say 0 or 1 
                  max_epochs=30,
                  logger=logger,
                  callbacks=[EarlyStopping(monitor="val_loss", patience=6, mode='min'),
                             checkpoint_callback])

Global seed set to 142
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


CNN_classifier_3_layers_with_max_pooling_all_transforms_batch_norm_batch_size_82_lr_2e-4


In [ ]:
### Train and eval model

trainer.fit(model, train_loader, test_loader)  # here we are using test set as validation set
trainer.test(ckpt_path="best", dataloaders=test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.6/dist-packages/OpenSSL/crypto.py:8: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography import utils, x509

  | Name      | Type       | Params
-----------------------------------------
0 | AUROC     | AUROC      | 0     
1 | Precision | Precision  | 0     
2 | Recall    | Recall     | 0     
3 | model     | Sequential | 2.4 M 
-----------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.591     Total estimated model params size (MB)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /alloc/data/fury_fda-fraud-evasion-off/notebooks/lightning_checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exis

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/usr/local/lib/python3.6/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
Global seed set to 142
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) i

Training: 0it [00:00, ?it/s]

'NoneType' object is not iterable
> <ipython-input-9-dfeffc0812a7>(38)__call__()
     37             import ipdb; ipdb.set_trace()
---> 38             pass
     39         return img, bbox



...

3


ipdb>  c


'NoneType' object is not iterable
> <ipython-input-9-dfeffc0812a7>(38)__call__()
     37             import ipdb; ipdb.set_trace()
---> 38             pass
     39         return img, bbox



ipdb>  c


'NoneType' object is not iterable
> <ipython-input-9-dfeffc0812a7>(38)__call__()
     37             import ipdb; ipdb.set_trace()
---> 38             pass
     39         return img, bbox



### Eval previously trained model

checkpoint_callback.best_model_path = '/alloc/data/fury_fda-fraud-evasion-off/notebooks/lightning_checkpoints/test_model_training-v8.ckpt'

best_model = model.load_from_checkpoint(checkpoint_callback.best_model_path, **model_kwargs).to(device)
trainer.test(model=best_model, dataloaders=test_loader, verbose=True)

